In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse
import statsmodels.api as sm
from sklearn.covariance import EllipticEnvelope
import math
import datetime
import warnings
warnings.filterwarnings("ignore")

In [ ]:
crash_data= pd.read_excel('/content/sample_data/test_B_9_50.xlsx')

In [ ]:
residuals = []

In [ ]:
crash_data.dropna(inplace = True)
stocks = crash_data.TICKER.unique()

Original:
for i in stocks:    
    Y = crash_data.loc[crash_data['TICKER'] == i]['Ret'].values
    X = crash_data.loc[crash_data['TICKER'] == i]['vwretx'].values
    X = sm.add_constant(X)
    ols = sm.OLS(Y[2:-2], X[2:-2] + X[1:-3] + X[0:-4] +
                 X[3:-1] + X[4:]).fit()
    residuals.append(ols.resid)

In [ ]:
for i in stocks:
    df = crash_data.loc[crash_data['TICKER'] == i]
    df.fillna(df.mean(), inplace=True)
    Y = df['Ret'].values
    X = df['vwretx'].values
    X = sm.add_constant(X)
    ols = sm.OLS(Y[2:-2], X[2:-2] + X[1:-3] + X[0:-4] +
                 X[3:-1] + X[4:]).fit()
    residuals.append(ols.resid)


In [ ]:
residuals = list(map(lambda x: np.log(1+x), residuals))


In [ ]:
crash_data_sliced = pd.DataFrame ([])
for i in stocks:
    crash_data_sliced = crash_data_sliced.append(crash_data.loc[crash_data.TICKER == i]
                                                 [2:-2])
crash_data.replace([np.inf, -np.inf], np.nan, inplace=True)
crash_data_sliced.head()

,Date,Adj Close,PRC,ASKHI,BIDLO,Open,VOL,Ret,vwretx,TICKER
368,2012-01-18,0.153956,0.185,0.190,0.185,0.185,2506300.0,-2.631576,-0.178602,1481.KL
369,2012-01-25,0.162278,0.195,0.195,0.185,0.185,3545300.0,5.405400,0.358421,1481.KL
370,2012-02-01,0.166439,0.200,0.205,0.185,0.190,9528000.0,2.564108,0.159561,1481.KL
371,2012-02-08,0.178922,0.215,0.220,0.200,0.200,23597700.0,7.500000,0.441783,1481.KL
372,2012-02-15,0.170600,0.205,0.220,0.200,0.215,17489800.0,-4.651165,-0.278895,1481.KL


In [ ]:
#tambahan dari GPT
# Check for and remove NaN and inf values
for i, res in enumerate(residuals):
    mask = np.logical_not(np.isnan(res)) & np.logical_not(np.isinf(res))
    valid_values = res[mask]


In [ ]:
#tambahan dari GPT
# Replace values outside the range with the mean
residuals[i] = np.where(np.abs(residuals[i]) > np.finfo(np.float64).max, np.mean(residuals[i]), residuals[i])

In [ ]:
print(residuals)

In [ ]:
envelope = EllipticEnvelope(contamination=0.02, support_fraction=1)
ee_prediction = {}
for i, j in zip(range(len(stocks)), stocks):
    # Check for and remove NaN values
    mask = ~np.isnan(residuals[i])
    valid_values = residuals[i][mask]

    # Fit the model with the valid values
    envelope.fit(np.array(valid_values).reshape(-1, 1))
    ee_prediction[j] = envelope.predict(np.array(valid_values).reshape(-1,1))


In [ ]:
transform = []
for i in stocks:
    df = crash_data.loc[crash_data['TICKER'] == i]
    df.fillna(df.mean(), inplace=True)
    Y = df['Ret'].values
    X = df['vwretx'].values
    X = sm.add_constant(X)
    ols = sm.OLS(Y[2:-2], X[2:-2] + X[1:-3] + X[0:-4] +
                 X[3:-1] + X[4:]).fit()
    if ols.resid.shape[0] > 0:  # Only append non-zero dimensional arrays
        transform.append(envelope.predict(np.array(ols.resid).reshape(-1,1)))


In [ ]:
#transform = []

In [ ]:
#for i in stocks:
 #   for j in range(len(ee_prediction[i])):
  #      transform.append(np.where(ee_prediction[i][j] == 1, 0, -1))


In [ ]:
crash_data_sliced = crash_data_sliced.reset_index()
crash_data_sliced['residuals'] = np.concatenate(residuals)
crash_data_sliced['neg_outliers'] = np.where(np.concatenate(transform) == -1, 1, 0)
crash_data_sliced.loc[(crash_data_sliced.neg_outliers == 1) & (crash_data_sliced.residuals > 0), 'neg_outliers'] = 0

In [ ]:
crash_data_sliced = crash_data_sliced.set_index('Date')
crash_data_sliced.index = pd.to_datetime(crash_data_sliced.index)

In [ ]:
std= crash_data.groupby('TICKER')['Ret'].std().reset_index()
crash_data['std'] = pd.DataFrame(std['Ret'])

In [ ]:
crash_data_sliced.head()

,index,Adj Close,PRC,ASKHI,BIDLO,Open,VOL,Ret,vwretx,TICKER,residuals,neg_outliers
Date,,,,,,,,,,,,
2012-01-18,368,0.153956,0.185,0.190,0.185,0.185,2506300.0,-2.631576,-0.178602,1481.KL,NaN,1
2012-01-25,369,0.162278,0.195,0.195,0.185,0.185,3545300.0,5.405400,0.358421,1481.KL,1.457933,0
2012-02-01,370,0.166439,0.200,0.205,0.185,0.190,9528000.0,2.564108,0.159561,1481.KL,0.599938,0
2012-02-08,371,0.178922,0.215,0.220,0.200,0.200,23597700.0,7.500000,0.441783,1481.KL,1.893864,0
2012-02-15,372,0.170600,0.205,0.220,0.200,0.215,17489800.0,-4.651165,-0.278895,1481.KL,NaN,1


In [ ]:
yearly_data = crash_data_sliced.groupby('TICKER')['residuals'].resample('Y').agg(['mean', 'std']).reset_index()
yearly_data.columns = ['TICKER', 'Date', 'mean', 'std']
yearly_data.head()

,TICKER,Date,mean,std
0,1481.KL,2012-12-31,0.619744,1.119262
1,1481.KL,2013-12-31,0.929528,1.222824
2,1481.KL,2014-12-31,1.020159,1.230626
3,1481.KL,2015-12-31,1.020520,1.396692
4,1481.KL,2016-12-31,1.028220,0.736725


In [ ]:
merge_crash = pd.merge(crash_data_sliced.reset_index(), yearly_data, how='outer', on=['TICKER', 'Date'])

In [ ]:
merge_crash[['annual_mean', 'annual_std']] = merge_crash.sort_values(by=['TICKER', 'Date']).iloc[:, -2:].fillna(method='bfill')
merge_crash['residuals'] = merge_crash.sort_values(by=['TICKER', 'Date'])['residuals'].fillna(method='ffill')
merge_crash = merge_crash.drop(merge_crash.iloc[: , -4: -2], axis = 1)

In [ ]:
crash_risk_out=[]
for j in stocks:
    for k in range(len(merge_crash[merge_crash.TICKER == j])):
        if merge_crash[merge_crash.TICKER == j]['residuals'].iloc[k]<merge_crash[merge_crash.TICKER == j]['annual_mean'].iloc[k]-3.09 * merge_crash[merge_crash.TICKER == j]['annual_std'].iloc[k]:crash_risk_out.append(1)
        else:
            crash_risk_out.append(0)

In [ ]:
merge_crash['crash_risk']=crash_risk_out
merge_crash['crash_risk'].value_counts()

0    16247
1      123
Name: crash_risk, dtype: int64

In [ ]:
merge_crash = merge_crash.set_index('Date')
merge_crash_annual = merge_crash.groupby('TICKER').resample('1Y')['crash_risk'].sum().reset_index()

In [ ]:
down = []

for j in range(len(merge_crash)):
    if merge_crash['residuals'].iloc[j]<merge_crash['annual_mean'].iloc[j]:
        down.append(1)
    else:
        down.append(0)

In [ ]:
merge_crash = merge_crash.reset_index()
merge_crash['down']=pd.DataFrame(down)
merge_crash['up'] = 1 - merge_crash['down']
down_residuals = merge_crash[merge_crash.down == 1][['residuals', 'TICKER','Date']]
up_residuals=merge_crash[merge_crash.up ==1][['residuals','TICKER','Date']]

In [ ]:
down_residuals['residuals_down_sq']=down_residuals['residuals'] ** 2
down_residuals['residuals_down_cubic']=down_residuals['residuals'] ** 3
up_residuals['residuals_up_sq']=up_residuals['residuals'] ** 2
up_residuals['residuals_up_cubic']=up_residuals['residuals'] ** 3
down_residuals['down_residuals']=down_residuals['residuals']
up_residuals['up_residuals']=up_residuals['residuals']
del down_residuals['residuals']
del up_residuals['residuals']

In [ ]:
merge_crash['residuals_sq']=merge_crash['residuals']**2
merge_crash['residuals_cubic']=merge_crash['residuals']**3

In [ ]:
merge_crash_all = merge_crash.merge(down_residuals, on=['TICKER', 'Date'], how='outer')
merge_crash_all = merge_crash.merge(up_residuals, on=['TICKER', 'Date'], how='outer')
merge_crash_all = pd.merge(merge_crash_all, down_residuals[['TICKER', 'Date', 'residuals_down_sq']], on=['TICKER', 'Date'], how='left') #additionalfrom GPT

In [ ]:
merge_crash_all.shape

(16370, 24)

In [ ]:
pd.set_option('display.max_columns', 24)
merge_crash_all.head()

,Date,index,Adj Close,PRC,ASKHI,BIDLO,Open,VOL,Ret,vwretx,TICKER,residuals,neg_outliers,annual_mean,annual_std,crash_risk,down,up,residuals_sq,residuals_cubic,residuals_up_sq,residuals_up_cubic,up_residuals,residuals_down_sq
0,2012-01-18,368.0,0.153956,0.185,0.190,0.185,0.185,2506300.0,-2.631576,-0.178602,1481.KL,NaN,1.0,0.619744,1.119262,0,0,1,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-01-25,369.0,0.162278,0.195,0.195,0.185,0.185,3545300.0,5.405400,0.358421,1481.KL,1.457933,0.0,0.619744,1.119262,0,0,1,2.125568,3.098935,2.125568,3.098935,1.457933,NaN
2,2012-02-01,370.0,0.166439,0.200,0.205,0.185,0.190,9528000.0,2.564108,0.159561,1481.KL,0.599938,0.0,0.619744,1.119262,0,1,0,0.359925,0.215933,NaN,NaN,NaN,0.359925
3,2012-02-08,371.0,0.178922,0.215,0.220,0.200,0.200,23597700.0,7.500000,0.441783,1481.KL,1.893864,0.0,0.619744,1.119262,0,0,1,3.586720,6.792759,3.586720,6.792759,1.893864,NaN
4,2012-02-15,372.0,0.170600,0.205,0.220,0.200,0.215,17489800.0,-4.651165,-0.278895,1481.KL,1.893864,1.0,0.619744,1.119262,0,0,1,3.586720,6.792759,3.586720,6.792759,1.893864,NaN


In [ ]:
cols=['BIDLO', 'ASKHI', 'residuals', 'annual_std', 'residuals_sq', 'residuals_cubic', 'down', 'up', 'residuals_up_sq', 'residuals_down_sq','neg_outliers']
merge_crash_all= merge_crash_all.set_index('Date')
merge_grouped = merge_crash_all.groupby('TICKER')[cols].resample('1Y').sum().reset_index()
merge_grouped['neg_outliers']=np.where(merge_grouped.neg_outliers>= 1, 1, 0)

In [ ]:
merge_grouped=merge_grouped.set_index('Date')
merge_all = merge_grouped.groupby('TICKER').resample('1Y').agg({'down':['sum', 'count'], 'up':['sum', 'count']}).reset_index()
merge_all.head()

TICKER       Date down        up      
                       sum count sum count
0  1481.KL 2012-12-31   18     1  33     1
1  1481.KL 2013-12-31   17     1  36     1
2  1481.KL 2014-12-31   24     1  29     1
3  1481.KL 2015-12-31   15     1  38     1
4  1481.KL 2016-12-31   23     1  29     1

In [ ]:
merge_grouped['down']=merge_all['down']['sum'].values
merge_grouped['up']=merge_all['up']['sum'].values
merge_grouped['count']=merge_grouped['down'] + merge_grouped['up']

In [ ]:
merge_grouped = merge_grouped.reset_index()

In [ ]:
merge_grouped['duvol']=np.log(((merge_grouped['up']-1)* merge_grouped['residuals_down_sq'])/((merge_grouped['down']- 1) * merge_grouped['residuals_up_sq']))

In [ ]:
merge_grouped['duvol'].mean()

-0.9057143197974393

In [ ]:
merge_grouped['ncskew'] = - (((merge_grouped['count']*(merge_grouped['count'] -1)** (3/2))* merge_grouped['residuals_cubic'])/((merge_grouped['count']-1) * merge_grouped['residuals_sq']**(3/2)))

In [ ]:
merge_grouped['duvol']

0     -0.454451
1     -0.754910
2     -1.617109
3     -0.626657
4     -2.099582
         ...   
316   -2.244220
317   -1.346018
318   -0.967489
319   -0.688195
320   -1.599988
Name: duvol, Length: 321, dtype: float64

In [ ]:
merge_grouped['ncskew'].mean()

-31.416120599338655

In [ ]:
merge_grouped['crash_risk']= merge_crash_annual['crash_risk']
merge_grouped['crash_risk'] =np.where(merge_grouped.crash_risk>=1, 1, 0)

In [ ]:
merge_crash_all_grouped2 =merge_crash_all.groupby('TICKER')[['VOL', 'PRC']].resample('1Y').mean().reset_index()
merge_grouped[['VOL', 'PRC']] = merge_crash_all_grouped2[['VOL','PRC']]
merge_grouped[['ncskew','duvol']].corr()

,ncskew,duvol
ncskew,1.000000,0.909498
duvol,0.909498,1.000000


In [ ]:
merge_crash_all_grouped2.shape

(321, 4)

In [ ]:
merge_crash_all_grouped2

,TICKER,Date,VOL,PRC
0,1481.KL,2012-12-31,5.749518e+06,0.168200
1,1481.KL,2013-12-31,4.036952e+06,0.143173
2,1481.KL,2014-12-31,1.758116e+07,0.177736
3,1481.KL,2015-12-31,3.779940e+06,0.139327
4,1481.KL,2016-12-31,2.156496e+06,0.114608
...,...,...,...,...
316,9814.KL,2014-12-31,3.411691e+06,0.829151
317,9814.KL,2015-12-31,3.469600e+05,0.636600
318,9814.KL,2016-12-31,2.790490e+05,0.539694
319,9814.KL,2017-12-31,2.842353e+05,0.489118


In [ ]:
merge_grouped

,Date,TICKER,BIDLO,ASKHI,residuals,annual_std,residuals_sq,residuals_cubic,down,up,residuals_up_sq,residuals_down_sq,neg_outliers,count,duvol,ncskew,crash_risk,VOL,PRC
0,2012-12-31,1481.KL,8.200,8.760,37.645451,57.082351,90.182232,68.980438,18,33,67.439301,22.742930,1,51,-0.454451,-29.046891,0,5.749518e+06,0.168200
1,2013-12-31,1481.KL,7.170,7.715,57.795494,64.809687,124.551419,129.884869,17,36,102.521470,22.029949,1,53,-0.754910,-35.711910,1,4.036952e+06,0.143173
2,2014-12-31,1481.KL,8.975,9.890,59.852542,65.223195,137.450517,245.454712,24,29,118.183096,19.267421,1,53,-1.617109,-58.214255,0,1.758116e+07,0.177736
3,2015-12-31,1481.KL,6.935,7.550,66.632232,74.024682,159.403128,182.061673,15,38,132.593290,26.809839,1,53,-0.626657,-34.574094,1,3.779940e+06,0.139327
4,2016-12-31,1481.KL,5.690,6.090,54.671805,38.309722,86.063723,140.082671,23,29,78.506960,7.556763,1,52,-2.099582,-65.154235,0,2.156496e+06,0.114608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316,2014-12-31,9814.KL,42.210,45.685,58.997619,49.895256,115.847342,219.994617,23,30,107.224210,8.623132,1,53,-2.244220,-67.431137,0,3.411691e+06,0.829151
317,2015-12-31,9814.KL,30.610,33.095,44.818273,66.776944,112.720006,169.921418,20,31,96.768607,15.951399,1,51,-1.346018,-51.203754,0,3.469600e+05,0.636600
318,2016-12-31,9814.KL,25.045,27.075,51.531091,64.647495,131.912025,172.918858,20,30,105.615030,26.296995,1,50,-0.967489,-39.946968,0,2.790490e+05,0.539694
319,2017-12-31,9814.KL,24.450,25.700,36.160650,61.513834,85.939285,69.806990,20,32,65.704180,20.235105,1,52,-0.688195,-32.538666,0,2.842353e+05,0.489118


In [ ]:
data_mining = pd.DataFrame(merge_grouped[['Date', 'TICKER', 'duvol', 'ncskew']])

In [ ]:
data_mining

,Date,TICKER,duvol,ncskew
0,2012-12-31,1481.KL,-0.454451,-29.046891
1,2013-12-31,1481.KL,-0.754910,-35.711910
2,2014-12-31,1481.KL,-1.617109,-58.214255
3,2015-12-31,1481.KL,-0.626657,-34.574094
4,2016-12-31,1481.KL,-2.099582,-65.154235
...,...,...,...,...
316,2014-12-31,9814.KL,-2.244220,-67.431137
317,2015-12-31,9814.KL,-1.346018,-51.203754
318,2016-12-31,9814.KL,-0.967489,-39.946968
319,2017-12-31,9814.KL,-0.688195,-32.538666


In [ ]:
data_mining.to_excel(r'/content/sample_data/data titipan dari python.xlsx', index=False)